In [0]:

%run /Workspace/Shared/gu_census_crawl/common_crawl/src/helper_functions_new  

In [0]:
aws_access_key_id = dbutils.secrets.get(scope='aws_cc', key='aws_access_key_id')
aws_secret_access_key = dbutils.secrets.get(scope='aws_cc', key='aws_secret_access_key')

In [0]:
# Optional: build client once (faster)
s3 = boto3.client(
    "s3",
    region_name="us-east-1",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [0]:
most_recent_crawl = (
    spark.read.table("census_bureau_capstone.silver.cleaned_master_crawls_2025")
    .orderBy('master_index', ascending=False)
    .select('master_index')
    .limit(1) \
    .collect()[0]['master_index']
)

display(most_recent_crawl)

In [0]:
df_recent_crawl = cc.list_crawls(most_recent_crawl)

In [0]:
from pyspark.sql.functions import lit
df_recent_crawl = spark.createDataFrame(df_recent_crawl).withColumn("crawl_prefix", lit(most_recent_crawl))

In [0]:
total_rows = df_recent_crawl.count()
print(f'Total rows in most recent crawl: {total_rows}')

In [0]:
from pyspark.sql.functions import col

duplicates_key = (
    df_recent_crawl
    .groupBy("Key")
    .count()
    .filter(col("count") > 1)
    .count()
)

print(f'Total duplicate rows in most recent crawl: {duplicates_key}')

In [0]:
df_bronze = spark.read.table("census_bureau_capstone.bronze.raw_all_crawls")

In [0]:
df_new = (
    df_recent_crawl
    .join(
        df_bronze.select("Key"),
        on="Key",
        how="left_anti"
    )
)

df_bronze_updated = df_bronze.unionByName(df_new)

In [0]:
df_bronze_updated.write.mode("overwrite").saveAsTable("census_bureau_capstone.bronze.raw_all_crawls")